In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
import pandas as pd

In [19]:
# function to get urls from data scientist jobs in canada
# each page ends with start=50/start=100 etc, so start, end need to be in multiples of 50
def get_ds_urls(start, end): 
    urls = []
    # Preparing the monitoring of the loop
    start_time = time()
    requests = 0
  
    for i in range(start, end, 50):
        driver = webdriver.Chrome('C:/Users/nalgn/Documents/Insight/project/chromedriver')
        driver.get('https://resumes.indeed.com/search?l=canada&q=anytitle%3A%28data%20scientist%29&start='+str(i))      
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.close()                
        
        for result in soup.find_all('div', class_='rezemp-ResumeSearchCard'): 
            urls.append('https://resumes.indeed.com' + str(result.find('span').a['href']))
        
        # pause the loop             
        sleep(randint(4, 8))
        # monitor requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Time: {}; Frequency: {} requests/s'.format(requests, elapsed_time, requests/elapsed_time))
        clear_output(wait = True)
    
    return urls

In [23]:
ds_urls = get_ds_urls(0, 450)

Request:9; Time: 116.95661044120789; Frequency: 0.07695161450086781 requests/s


In [24]:
ds_urls

['https://resumes.indeed.com/resume/7f33ad1fa4c56629?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/f290c9b2f88e6713?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/402102545b124c75?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/d43b04bd4265fc96?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/235f6df1f5eab3d8?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/cf309dfc0b1e7487?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/7ae5e563aa553097?s=l%3Dcanada%26q%3Danytitle%253A%2528data%2520scientist%2529%26start%3D50',
 'https://resumes.indeed.com/resume/df08298a95fdb865?s=l%3Dcanada%26q%3Danytitle%253A%2528data%25

In [22]:
# function to get resume from one url
def get_resume(url):
    driver = webdriver.Chrome('C:/Users/nalgn/Documents/Insight/project/chromedriver')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.close()
      
    resume = soup.find_all('div', class_='rezemp-ResumeDisplaySection') # get all sections of resume
                                                                     
    # work experience
    job_titles = []
    job_durations = []
    job_descriptions = [] 
    try:
        work = resume[0] # first section of resume
        for work_exp in work.find_all('div', class_='rezemp-WorkExperience'):
            title = work_exp.span.text
            job_titles.append(title)
            duration = work_exp.find('div', class_='icl-u-textColor--tertiary').text
            job_durations.append(duration)
            description = work_exp.find_all('div')[4].text # 5th div under work experience
            job_descriptions.append(description)
    except:
        pass
    
    # education
    degrees = []
    schools = []
    graduation_dates = [] 
    try:
        education = resume[1]
        for ed in education.find('div', class_='rezemp-ResumeDisplaySection-content'):
            degree = ed.find('span', class_='rezemp-ResumeDisplay-itemTitle').text 
            degrees.append(degree)
            school = ed.find('div', class_='rezemp-ResumeDisplay-university')
            school_name = school.find('span', class_='icl-u-textBold').text
            schools.append(school_name)
            date = ed.find('span', class_='rezemp-ResumeDisplay-date').text
            graduation_dates.append(date)
    except:
        pass
    
    return job_titles, job_durations, job_descriptions, degrees, schools, graduation_dates

In [23]:
# function to get resumes from multiple urls
def get_multiple_resumes(url_list):
    resumes = []
    start_time = time()
    requests = 0
    for url in url_list:
        resume = get_resume(url)
        resumes.append(resume)
        
        # pause the loop             
        sleep(randint(4, 8))
        # monitor requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Time:{}; Frequency: {} requests/s'.format(requests, elapsed_time, requests/elapsed_time))
        clear_output(wait = True)
    
    resumes_df = pd.DataFrame(resumes, columns=['job_titles', 'job_durations', 'job_descriptions',
                                               'degrees', 'schools', 'graduation_dates'])
    
    return resumes_df    

In [41]:
ds_resumes = get_multiple_resumes(ds_urls_all)

Request:420; Time:5170.464958667755; Frequency: 0.0812306056336216 requests/s


In [42]:
ds_resumes

,job_titles,job_durations,job_descriptions,degrees,schools,graduation_dates
0,"[Data Scientist, Business Inteligence Consultant]","[February 2013 to Present, January 2006 to Feb...","[2016 ~ Manager, Smart Solution Team in Inform...","[Master in Business Administration, Bachelor o...","[KAIST, Konkuk University]","[January 2012 to January 2013, March 1999 to F..."
1,"[Director of Business Relations, Senior Data A...","[May 2018 to Present, May 2014 to July 2016]",[GIMME360 Data Scientist-Business Solution 2...,"[Master in applied mathematics, Bachelor in ap...","[University of Waterloo, University of Waterloo]","[January 2016 to January 2018, January 2010 to..."
2,"[FREELANCE DEVELOPER, Data Scientist, DESIGN E...","[March 2016 to Present, April 2018 to August 2...","[Toronto, Canada• Completed 30+ freelancer Pro...",[BACHELORS in COMPUTER ENGINEERING],[UNIVERSITY OF TORONTO],[April 2021]
3,"[Automation Test Engineer, Data Scientist, Dat...","[December 2018 to Present, August 2018 to Nove...",[Responsibilities: ⚫ GM Infotainment system...,[Master in Electrical and Computer Engineering...,"[University of Ottawa, Beijing Institute of Te...","[September 2015 to June 2017, September 2009 t..."
4,"[Validation Scientist Co-Op (PEY), Data Collec...","[September 2017 to August 2018, May 2015 to Se...",[• Developing qualification document for a new...,[B.A.Sc in Electrical Engineering],[University of Toronto],[September 2014 to April 2019]
5,"[Research Assistant, Data Scientist, Research ...","[May 2016 to Present, January 2018 to May 2018...","[• Developed skills in research design, data a...","[hBA in Psychology and Business, Research Spec...",[University of Waterloo],[May 2015 to May 2019]
6,"[Developer, Software Developer, Data Scientist...","[July 2018 to Present, December 2017 to June 2...",[QVC is working on implementing the NoSQL data...,"[M.S in Computer Science, B.S in Computer Scie...","[New Jersey Institute of Technology, Amrita Sc...","[January 2015, January 2013]"
7,"[Data Scientist, Machine Learning Engineer, Ma...","[January 2004 to Present, September 2015 to De...","[Industry Projects, · Automatic Analysis for H...","[Master of Science in Computer Science, Master...","[Memorial University of Newfoundland, Azad Uni...","[September 2015 to December 2018, October 2004..."
8,"[Machine learning Scientist, Machine Learning ...","[May 2018 to Present, October 2017 to Present,...",[• Using ML to automate aspects of graphic des...,"[MSc in parallel computing, BSc in Physics and...","[Trinity College Dublin, University College Cork]","[September 2008 to September 2009, January 200..."
9,"[Senior Data Scientist, Senior Advisory Consul...","[July 2018 to January 2019, April 2015 to July...","[Personalise prospect acquisition, member enga...","[PG Diploma in Data Science, MBA in Finance, ,...",[International Institute of Information Techno...,"[July 2017 to July 2018, July 2009 to May 2011..."


In [44]:
ds_resumes.to_csv('ds_resumes_scraped_1_23.csv', index=False)